# **Mountain Car (Discrete case)**

References :

SARSA:  https://github.com/nicklasbekkevold/SARSA-mountaincar

SARSA (lambda) : https://github.com/dariopavllo/mountaincar-sarsa-lambda

Semi-Gradient SARSA : https://github.com/MikeS96/rl_openai/blob/master/Mountain%20Car.ipynb
http://www.incompleteideas.net/book/RLbook2018.pdf#page=267


PPO:    https://github.com/rossettisimone/PPO_MOUNTAINCAR_DISCRETE.

Reddit: www.reddit.com/r/reinforcementlearning/comments/axp63j/d_state_of_the_art_deeprl_still_struggles_to/.  
https://arxiv.org/pdf/1802.05054


Documentation pour la librairie tiles3 : http://incompleteideas.net/tiles/tiles3.html


Q-learning : https://www.kaggle.com/code/rezafazel63/mountain-car-is-a-classic-reinforcement-learning


  

# Google Colab configs

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [1]:
!pip install gym
!pip install gym[classic_control]
!pip install pyvirtualdisplay

     ---------------------------------------- 0.0/721.7 kB ? eta -:--:--
     - ----------------------------------- 20.5/721.7 kB 217.9 kB/s eta 0:00:04
     -- ---------------------------------- 41.0/721.7 kB 393.8 kB/s eta 0:00:02
     ---------------- --------------------- 317.4/721.7 kB 2.2 MB/s eta 0:00:01
     -------------------------------- ----- 614.4/721.7 kB 3.2 MB/s eta 0:00:01
     -------------------------------------- 721.7/721.7 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached numpy-2.2.0-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached cloudpickle-3.1.0-py3-none-any.whl.metadata (7.0 kB)
Using cached cloudpickle-3.1.0-py3-none-any.whl (22 kB)
Usi


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/5.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.8 MB ? eta -:--:--
      --------------------------------------- 0.1/5.8 MB 573.4 kB/s eta 0:00:10
     -- ------------------------------------- 0.4/5.8 MB 2.4 MB/s eta 0:00:03
     -------- ------------------------------- 1.2/5.8 MB 5.4 MB/s eta 0:00:01
     ---------------- ----------------------- 2.4/5.8 MB 8.8 MB/s eta 0:00:01
     -------------------------- ------------- 3.9/5.8 MB 12.3 MB/s eta 0:00:01
     ---------------------------------- ----- 5.0/5.8 MB 14.0 MB/s eta 0:00:01
     ---------------------------------------  5.8/5.8 MB 16.1 MB/s eta 0:00:01
     ---------------------------------------- 5.8/5.8 MB 14.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Gettin

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [95 lines of output]
      
      
      WARNING, No "Setup" File Exists, Running "buildconfig/config.py"
      Using WINDOWS configuration...
      
      Making dir :prebuilt_downloads:
      Downloading... https://www.libsdl.org/release/SDL2-devel-2.0.16-VC.zip 13d952c333f3c2ebe9b7bc0075b4ad2f784e7584
      Unzipping :prebuilt_downloads\SDL2-devel-2.0.16-VC.zip:
      Downloading... https://www.libsdl.org/projects/SDL_image/release/SDL2_image-devel-2.0.5-VC.zip 137f86474691f4e12e76e07d58d5920c8d844d5b
      Unzipping :prebuilt_downloads\SDL2_image-devel-2.0.5-VC.zip:
      Downloading... https://www.libsdl.org/projects/SDL_ttf/release/SDL2_ttf-devel-2.0.15-VC.zip 1436df41ebc47ac36e02ec9bda5699e80ff9bd27
      Unzipping :prebuilt_downloads\SDL2_ttf-devel-2.0.15-VC.zip:
      Downloading... https://www.libsdl.org/projects/SDL_mixer/release/SDL2_mixer-devel-


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# !pip install gym pyvirtualdisplay > /dev/null 2>&1
# !apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
# !apt-get update > /dev/null 2>&1
# !apt-get install cmake > /dev/null 2>&1
# !pip install --upgrade setuptools 2>&1
# !pip install ez_setup > /dev/null 2>&1
# !apt-get install -y xvfb


def wrap_env(env):
    env = RecordVideo(env, "./video")
    return env


def show_video():
    mp4list = glob.glob("video/*.mp4")
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, "r+b").read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(
            HTML(
                data="""<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>""".format(
                    encoded.decode("ascii")
                )
            )
        )
    else:
        print("Could not find video")


# A executer avant l'initialisation de env
# display = Display(visible=0, size=(1400, 900))
# display.start()
# env = wrap_env(gym.make('MountainCar-v0'))

# Libraries

In [1]:
import numpy as np
import gymnasium as gym

# from gym.wrappers.record_video import RecordVideo
# import glob
# import io
# import base64
# from IPython.display import HTML
# from pyvirtualdisplay import Display
# from IPython import display as ipythondisplay

# 1. Functions & Algorithms

## 1.1. Utils

In [2]:
import math
import random


class IHT:
    def __init__(self, sizeval):
        self.size = sizeval
        self.overfullCount = 0
        self.dictionary = {}

    def __str__(self):
        return (
            "Collision table: size: "
            + str(self.size)
            + " overfullCount: "
            + str(self.overfullCount)
            + " dictionary: "
            + str(len(self.dictionary))
        )

    def count(self):
        return len(self.dictionary)

    def fullp(self):
        return len(self.dictionary) >= self.size

    def getindex(self, obj, readonly=False):
        if obj in self.dictionary:
            return self.dictionary[obj]
        elif readonly:
            return None
        size = self.size
        if len(self.dictionary) >= size:
            if self.overfullCount == 0:
                print("IHT full, starting to allow collisions")
            self.overfullCount += 1
            return hash(obj) % self.size
        else:
            self.dictionary[obj] = len(self.dictionary)
            return self.dictionary[obj]


def hashcoords(coordinates, m, readonly=False):
    if isinstance(m, IHT):
        return m.getindex(tuple(coordinates), readonly)
    if isinstance(m, int):
        return hash(tuple(coordinates)) % m
    if m is None:
        return coordinates


def get_tiles(ihtORsize, numtilings, floats, ints=[], readonly=False):
    qfloats = [int(math.floor(f * numtilings)) for f in floats]
    tiles = []
    for tiling in range(numtilings):
        tilingX2 = tiling * 2
        coords = [tiling]
        b = tiling
        for q in qfloats:
            coords.append((q + b) // numtilings)
            b += tilingX2
        coords.extend(ints)
        tiles.append(hashcoords(coords, ihtORsize, readonly))
    return tiles

In [3]:
POSITION_MIN = -1.2
POSITION_MAX = 0.5
VELOCITY_MIN = -0.07
VELOCITY_MAX = 0.07

iht_size = 2024
num_tilings = 8
num_tiles = 8

iht = IHT(2024)


def mc_tile_encoding(state):
    """
    Tile encoding function for states in mountain car gymnasium problem

    Args:
        - state (tuple): (position, velocity)

    Returns:
        - The list of the tiles corresponding to the given state
    """
    # Extract the position and velocity from the gymnasium state
    position, velocity = state

    # Scale position and velocity by multiplying the inputs of each by their scale
    position_scale = num_tiles / (POSITION_MAX - POSITION_MIN)
    velocity_scale = num_tiles / (VELOCITY_MAX - VELOCITY_MIN)

    # Obtain active tiles for current position and velocity
    tiles = get_tiles(
        iht, num_tilings, [position * position_scale, velocity * velocity_scale]
    )

    return np.array(tiles)


print("Tiles activées :", mc_tile_encoding((-1.0, 0.01)))

Tiles activées : [0 1 2 3 4 5 6 7]


## 1.2. Agents

### 1.2.1. SARSA Agent

In [27]:
class SarsaAgent:

    def __init__(
        self,
        env: gym.Env,
        encode_fct,
        nb_actions=None,
    ):
        """
        - env_name (gymnasium.Env): The environment to train on.
        - nb_actions (int): Number of possible actions.
        - encode_fct (callable): Function to encode the state into features (e.g., tile coding).
        """
        self.env = env
        self.nb_actions = env.action_space.n if nb_actions == None else nb_actions
        self.encode_fct = encode_fct
        self.q = {}

    def init(self):
        """Initialize q values"""
        self.q = {}

    def get_q_value(self, state, action):
        """Compute Q-value for a state-action pair by summing over the tiles."""
        return sum(self.q.get((tile, action), 0) for tile in self.encode_fct(state))

    def choose_action(self, state, epsilon=None, soft_policy=True):
        """
        Choose an action using an epsilon-greedy policy.

        Args:
        - q (dict): The Q-value dictionary.
        - state (hashable): The current state.
        - epsilon (float, optional): The probability of exploration (for soft policy).
        - soft_policy (bool, optional): If True, use epsilon-greedy; otherwise, choose the action with the highest Q-value.

        Returns:
        - int: The selected action.

        Raises:
        - ValueError: If soft_policy is True and epsilon is None.
        """
        if (soft_policy) and (epsilon is None):
            raise ValueError("Epsilon must be specified when soft_policy is True.")

        if (soft_policy) and (np.random.rand() < epsilon):
            return np.random.choice(self.nb_actions)  # Exploration
        return np.argmax(
            [self.get_q_value(state, action) for action in range(self.nb_actions)]
        )

    def train(
        self,
        nb_episodes,
        alpha=0.1,
        gamma=0.99,
        epsilon=0.1,
        use_glei=False,
        verbose=False,
    ):
        """
        SARSA algorithm for on-policy reinforcement learning.

        Args:
            - nb_episodes (int): Number of episodes to train for.
            - alpha (float): Learning rate for updating Q-values.
            - gamma (float): Discount factor for future rewards.
            - epsilon (float): Initial exploration rate for epsilon-greedy policy.
            - use_glei (bool): Whether to use a decaying epsilon (GLEI policy).
            - decay_rate (float): Decay rate for epsilon in GLEI policy.
            - min_epsilon (float): Minimum epsilon value in GLEI policy.
            - verbose (boolean): Print or not informations about training
        Returns:
            - rewards_historic (list): History of rewards across episodes.
        """
        rewads_historic = []
        self.init()

        for episode in range(nb_episodes):
            state, _ = self.env.reset()
            action = self.choose_action(state, epsilon=epsilon)

            task_completed, episode_over = False, False
            total_reward = 0

            while not (task_completed or episode_over):
                next_state, reward, task_completed, episode_over, _ = self.env.step(
                    action
                )
                next_action = self.choose_action(next_state, epsilon=epsilon)

                # Compute the SARSA update
                q_current = self.get_q_value(state, action)
                q_next = (
                    self.get_q_value(next_state, next_action)
                    if not (task_completed or episode_over)
                    else 0
                )
                target = reward + gamma * q_next
                error = target - q_current

                # Update Q-values for all tiles
                for tile in self.encode_fct(state):
                    self.q[(tile, action)] = (
                        self.q.get((tile, action), 0) + alpha * error
                    )

                # Move to the next state and action
                state = next_state
                action = next_action
                total_reward += reward

            rewads_historic.append(total_reward)
            if verbose:
                print(f"Episode {episode + 1}: Total Reward = {total_reward}")

        return rewads_historic

    #         Train the agent multiple times in order to evaluate its training parameters with mean and standard deviation over episodes

    def evaluate(
        self, num_episodes=10, soft_policy=False, render_mode=None, verbose=False
    ):
        """
        Evaluate the current q values of the agent and print the average reward

        Args:
            -

        Returns:
            - avg_reward (float): the average reward on the given evaluated period
        """
        if verbose
        env = gym.make("MountainCar-v0", render_mode=render_mode)
        total_rewards = []

        for episode in range(num_episodes):
            state, _ = env.reset()
            time_over = False
            done = False
            total_reward = 0

            while not (done or time_over):

                # Obtenir les tiles pour l'état actuel
                state_tiles = mc_tile_encoding(state)

                # Had policy by default for evaluation
                action = self.choose_action(state, soft_policy=soft_policy)

                # Exécuter l'action choisie
                state, reward, done, time_over, _ = env.step(action)
                # position, velocity = next_state

                total_reward += reward

            total_rewards.append(total_reward)
            if verbose:
                print(f"Episode {episode + 1}: Total Reward = {total_reward}")

        env.close()
        print(
            f"Average Total Reward over {num_episodes} episodes: {np.mean(total_rewards)}"
        )

    # def plot_moving_averages(rewards_dict, nb_episodes, moving_avg_size):
    # """
    # Plots the moving averages of rewards for multiple reward histories on the same graph,
    # with named histories provided in a dictionary.

    # Args:
    #     - rewards_dict (dict): A dictionary where keys are names of the reward histories
    #       and values are the corresponding lists of rewards.
    #     - moving_avg_size (int, optional): The size of the moving average window. Default is 20.

    # Returns:
    #     None
    # """
    # plt.figure(figsize=(12, 8))

    # # Loop through each rewards history in the dictionary and compute the moving average
    # for name, rewards in rewards_dict.items():

    #     # Calculate the moving average by sliding the kernel k = [1/moving_avg_size] * moving_avg_size
    #     moving_avg_rewards = np.convolve(rewards, np.ones(moving_avg_size) / moving_avg_size, mode='valid')

    #     plt.plot(
    #         range(len(moving_avg_rewards)),
    #         moving_avg_rewards,
    #         label=name
    #     )

    # # Add labels, title, legend, and grid
    # plt.xlabel("Number of Episodes - Moving Average Window Size")
    # plt.ylabel("Moving Average of Rewards")
    # plt.title(f"Comparison of {moving_avg_size}-Episode Moving Averages Across Reward Histories (trained on {nb_episodes} episodes)")
    # plt.legend()
    # plt.grid()
    # plt.show()

# 2.1. Experimentations

In [29]:
v0_agent = SarsaAgent(env=gym.make("MountainCar-v0"), encode_fct=mc_tile_encoding)

rewards_historic = v0_agent.train(
    nb_episodes=1000, alpha=0.1, gamma=0.99, epsilon=0.001
)
v0_agent.evaluate(verbose=True)

Episode 1: Total Reward = -106.0
Episode 2: Total Reward = -89.0
Episode 3: Total Reward = -105.0
Episode 4: Total Reward = -153.0
Episode 5: Total Reward = -147.0
Episode 6: Total Reward = -106.0
Episode 7: Total Reward = -141.0
Episode 8: Total Reward = -105.0
Episode 9: Total Reward = -139.0
Episode 10: Total Reward = -147.0
Average Total Reward over 10 episodes: -123.8


In [70]:
def test_policy(Q, num_episodes=10, render=False):
    render = "human" if render else None
    env = gym.make("MountainCar-v0", render_mode=render)
    total_rewards = []

    for episode in range(num_episodes):
        state = env.reset()[0]
        time_over = False
        done = False
        total_reward = 0

        while not (done or time_over):

            # Obtenir les tiles pour l'état actuel
            state_tiles = mc_tile_encoding(state)

            # Choisir l'action qui maximise Q(s, a)
            q_values = [
                sum(Q.get((tile, action), 0) for tile in state_tiles)
                for action in range(env.action_space.n)
            ]
            action = np.argmax(q_values)

            # Exécuter l'action choisie
            state, reward, done, time_over, _ = env.step(action)
            # position, velocity = next_state

            total_reward += reward

        total_rewards.append(total_reward)
        print(f"Episode {episode + 1}: Total Reward = {total_reward}")

    env.close()
    print(
        f"Average Total Reward over {num_episodes} episodes: {np.mean(total_rewards)}"
    )

In [71]:
# Tester la politique optimale
test_policy(Q, num_episodes=10, render=False)

Episode 1: Total Reward = -105.0
Episode 2: Total Reward = -105.0
Episode 3: Total Reward = -92.0
Episode 4: Total Reward = -106.0
Episode 5: Total Reward = -105.0
Episode 6: Total Reward = -178.0
Episode 7: Total Reward = -96.0
Episode 8: Total Reward = -93.0
Episode 9: Total Reward = -105.0
Episode 10: Total Reward = -105.0
Average Total Reward over 10 episodes: -109.0


In [72]:
# Tester la politique optimale
test_policy(v0_agent.q, num_episodes=10, render=True)

Episode 1: Total Reward = -107.0
Episode 2: Total Reward = -107.0
Episode 3: Total Reward = -95.0
Episode 4: Total Reward = -159.0
Episode 5: Total Reward = -107.0
Episode 6: Total Reward = -96.0
Episode 7: Total Reward = -90.0


2024-12-28 16:54:34.489 Python[13207:3704629] Warning: Window move completed without beginning


Episode 8: Total Reward = -178.0
Episode 9: Total Reward = -108.0
Episode 10: Total Reward = -87.0
Average Total Reward over 10 episodes: -113.4


In [73]:
env = gym.make(
    "MountainCar-v0",
    render_mode="human",  # Cas discret
)

env.reset()
print(f"State space: {env.observation_space}")
print(f"Action space: {env.action_space}")

State space: Box([-1.2  -0.07], [0.6  0.07], (2,), float32)
Action space: Discrete(3)


In [74]:
sum = 0
for i in range(201):
    print(i)
    a = env.step(2)
    sum += a[1]
    print(a)
    print(sum)

0
(array([-0.49266914,  0.00077414], dtype=float32), -1.0, False, False, {})
-1.0
1
(array([-0.49112663,  0.0015425 ], dtype=float32), -1.0, False, False, {})
-2.0
2
(array([-0.48882726,  0.00229935], dtype=float32), -1.0, False, False, {})
-3.0
3
(array([-0.48578823,  0.00303903], dtype=float32), -1.0, False, False, {})
-4.0
4
(array([-0.48203218,  0.00375606], dtype=float32), -1.0, False, False, {})
-5.0
5
(array([-0.47758707,  0.00444512], dtype=float32), -1.0, False, False, {})
-6.0
6
(array([-0.47248593,  0.00510113], dtype=float32), -1.0, False, False, {})
-7.0
7
(array([-0.46676666,  0.00571928], dtype=float32), -1.0, False, False, {})
-8.0
8
(array([-0.46047154,  0.0062951 ], dtype=float32), -1.0, False, False, {})
-9.0
9
(array([-0.45364708,  0.00682447], dtype=float32), -1.0, False, False, {})
-10.0
10
(array([-0.4463434 ,  0.00730368], dtype=float32), -1.0, False, False, {})
-11.0
11
(array([-0.43861398,  0.00772943], dtype=float32), -1.0, False, False, {})
-12.0
12
(array([

In [75]:
sum

-201.0

### Code Pierre

In [76]:
def on_policy_monte_carlo(
    env,
    nb_states,
    nb_actions,
    encode_fct,
    nb_episodes=20000,
    gamma=1,
    epsilon=0.1,
    use_glei=False,
    decay_rate=0.2,
    min_epsilon=0.05,
):
    """
    This algorithm goal is to find the optimal policy in a episodic environement with reasonable number of states and actions
    Convergence guaranteed in this paper: https://sites.ualberta.ca/~szepesva/papers/sarsa98.ps.pdf

    Repeat for the given number of episodes:
        Simulate an episode with the given policy
        Compute the accumulated rewards with discount factor (gamma): G
        For every visited q(s, a) in the current episode:
            If first visit of (s, a) is at iteration t:
                acc(s, a) += Gt
                count(s, a) += 1
                q(s, a) = acc(s, a) / count(s, a)
            For every state in the current episode:
                update policy(s, a) with an epsilon greedy strategy or GLEI strategy

    Args:
        - env (gymnasium.Env): a gymnasium environment
        - nb_states (int): the number of states for the given env
        - nb_actions (int): the number of actions for the given env
        - encode_fct (callable): the function to encode our state.
        - nb_episodes (int): number of episodes simulated for learning
        - gamma (float): discount factor for updating states values
        - epsilon (float): initial exploration factor, in [0, 1], if use_gley = True, represent the starting epsilon
        - use_glei (bool): if True, use GLEI policy (decaying epsilon across episodes); if False, use simple epsilon-greedy
        - decay_rate (float): Every time num_episode > nb_episode * (decay_rate *i) then epsilon = epsilon * (1 - decay_rate) * i. Where i is the number of time we decayed epsilon
        - min_epsilon (float): the minimum decayed epsilon when using a GLEI policy

    Returns:
        - policy (numpy.array): the learnt policy
        - q (numpy.array): the Q-values
        - rewards_historic (list): the historic of rewards during training
    """
    # Initialize algorithm variables
    q = np.zeros(
        shape=(nb_states, nb_actions)
    )  # (state, action) values through every episodes generated
    count = np.zeros(
        shape=(nb_states, nb_actions)
    )  # Counting each (s, a) events through every episodes generated
    acc = np.zeros(
        shape=(nb_states, nb_actions)
    )  # (state, action) cumulated values through every episodes generated
    policy = np.full(
        shape=(nb_states, nb_actions), fill_value=1 / nb_actions
    )  # Actions probabilities for each state. Initialized as a uniform policy
    rewards_historic = []
    count_decay = 1  # Decay rate factor

    for num_episode in range(nb_episodes):
        # print(f"Computing ep {num_episode}")

        # Initialize environment
        game_state = env.reset()[
            0
        ]  # A tuple (player score, opponent score, ace (1) or not (0))
        terminated = False
        G = np.empty((0, 1))  # Cumulated rewards of the current episode
        moves_historic = np.empty(
            (0, 2), dtype=int
        )  # Historic of (state, action) of the current episode

        # Step 1: Simulate an episode
        while not terminated:
            # Draw an action according to the current policy
            state_index = encode_fct(game_state)
            action = draw(policy, state_index)

            # Update environment with the chosen action
            env_state = env.step(action)
            game_state, G, terminated = (
                env_state[0],
                np.append(G, env_state[1]),
                env_state[2],
            )

            # Update current historic if current (state, action) was not visited yet
            moves_historic = np.append(moves_historic, [[state_index, action]], axis=0)

        # Update rewards_historic with the total reward of the episode
        rewards_historic.append(np.sum(G))

        # Step 2: Compute G, the cumulative rewards for each state
        gamma_powers = gamma ** np.arange(len(G))
        for i in range(len(G) - 1):
            G[i] = np.sum(G[i:] * gamma_powers[: len(G[i:])])

        # Step 3: For every (s, a) visited: Update q(s, a)
        # Extracting first visit for every (s, a)
        first_visit_states_action, first_visit_indices = np.unique(
            moves_historic, axis=0, return_index=True
        )
        first_visit_indices = np.sort(first_visit_indices)

        # Extract relevant states, actions, and rewards for first visits
        states = moves_historic[first_visit_indices, 0]
        actions = moves_historic[first_visit_indices, 1]
        rewards = G[first_visit_indices]

        # Update acc, count, and q arrays
        np.add.at(acc, (states, actions), rewards)  # Accumulate rewards
        np.add.at(count, (states, actions), 1)  # Increment count
        q = np.divide(acc, count, where=(count != 0), out=np.zeros_like(acc))

        # Step 4: Update policy with the current Q-values
        # Update epsilon if we use GLEI policy
        if (use_glei) & (num_episode > (decay_rate * count_decay) * nb_episodes):
            epsilon = max(epsilon * (1 - (decay_rate * count_decay)), min_epsilon)
            count_decay += 1
        policy = update_eps_greedy_policy(epsilon=epsilon, q=q)

    if use_glei:
        print(f"Final epsilon: {epsilon}")
    return policy, q, rewards_historic